In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.signal import stft
from scipy import signal
from tqdm import tqdm
import numpy as np
import random
import pickle
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow import keras

EPOCH = 100
KERNEL_SIZE = 3
BATCH_SIZE = 128
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512

DATA_PATH = "./mit_data/"
SAVE_PATH = "./spectrogram_plt/"

In [ ]:
parents_path = SAVE_PATH
child_path = os.listdir(parents_path)

temp_converted_img = list()
temp_ann_list = list()
converted_img = list()
X = np.array(list())
y = np.array(list())

for pic_path in (child_path):
    current_path = os.listdir(parents_path + pic_path)
    print("[INFO] Current path : " + parents_path + pic_path)
    for file_name in tqdm(current_path):
        path_for_array = parents_path + pic_path + "/" + file_name
        
        img_array = np.fromfile(path_for_array, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resize = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_AREA)
        temp_converted_img.append(img_resize / 255.0)
        
        check_ann = pic_path
        
        if check_ann == "N":            # Normal
            temp_ann_list.append(0)
        
        elif check_ann == "S":          # Supra-ventricular
            temp_ann_list.append(1)
        
        elif check_ann == "V":          # Ventricular
            temp_ann_list.append(2)
        
        elif check_ann == "F":          # False alarm
            temp_ann_list.append(3)
        
        else:                           # Unclassed 
            temp_ann_list.append(4)
    
    np.append(X, temp_converted_img)
    np.append(y, temp_ann_list)

In [ ]:
# 경로에 폴더가 없으면 폴더 만들기
def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42, shuffle=True )

npx = np.array(X_train)
npy = np.array(y_train)
npx_vali = np.array(X_val)
npy_vali = np.array(y_val)
npx_test = np.array(X_test)
npy_test = np.array(y_test)

print("[SIZE]\t\tNpX lenght : {}\n\t\tNpY length : {}".format(npx.shape, npy.shape))
print("[SIZE]\t\tX_validation length : {}\n\t\ty_validation length : {}".format(npx_vali.shape, npy_vali.shape))
print("[SIZE]\t\tX_test length : {}\n\t\ty_test length : {}".format(npx_test.shape, npy_test.shape))

In [ ]:
# print(X_train.unique)
# print(X_train.unique)

# class_names = y_train
# print(class_names)
# plt.figure(figsize=(10, 10))

# for images, labels in X_train.take(1):
#   for i in range(10):
#     ax = plt.subplot(5, 5, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(class_names[labels[i]])
#     plt.axis("off")

%matplotlib inline
plt.figure(figsize=(30, 12))
plt.suptitle("GOD DAMN", fontsize=18)
n = 0
random.seed(11)

for i in random.sample(range(6480), 16):
    ax = plt.subplot(5, 5, n+1)
    plt.plot(npx[i])
    ax.set_title(str(npy[i]))
    n+=1

In [ ]:
input_size = (256, 256, 1)

models = keras.Sequential([
    # tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(8, kernel_size=(4, 4), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(13, kernel_size=(2, 2), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Conv2D(13, kernel_size=(2, 2), activation='relu', input_shape=input_size),
    layers.MaxPool2D(pool_size=(2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')
])

In [ ]:
models.summary()

In [ ]:
# 콜백 설정
outDir = './cheakpoint/lefms_model/' # 이 경로에 best 모델이 저장된다.
model_names = outDir + 'weights-{val_accuracy:.4f}.h5'

def get_callbacks(patience = 50):
    earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=patience)
    model_checkpoint = ModelCheckpoint(model_names, monitor='val_accuracy', verbose=1, save_best_only=True, period = 1)

    callbacks = [model_checkpoint]
    return callbacks